**IMPORTANT** 

- For requirements and initial setup go to https://github.com/OliveiraEdu/OpenScience/Readme.md;
- To execute the notebook run all cells.

# Open Science Platform Notebook 2 - Project Account Creation

## Activities

1 - Deploys a smart contract into the Iroha 1 blockchain for account creation;

2 - Creates a project account into the blockchain;

3 - Queries the blockchain to confirm the succesful creation of the account;

4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

5 - Sets details for the project account;

6 - Queries the blockchain again to confirm the succesfull details setting.

Every account succesfully created is dumped to `datasets/projects.json` for later use and evidence collection.  

In [1]:
# from name_generator import left, right
from utilities import *
from project_id_generator import * # - Random generated synthetic project ID
from iroha_helper import *
from loguru import logger
from dump_to_json import ProjectAccount, update_or_append_project_metadata
from ipfs_functions import download_json_from_ipfs
from synthetic_project_json_ld import *
from new_helper import extract_account_metadata_cid_from_data, process_raw_data, load_variable
from loguru import logger

In [2]:
DOMAIN = "test"

1 - Loads the smart contract hash for account creation into the Iroha 1 blockchain.  

In [ ]:
# Loading the variable from JSON
create_account_contract_hash = load_variable("Create Account Hash")
logger.info(create_account_contract_hash)


2025-01-26 21:59:42,023 - INFO - Variable 'Create Account Hash' successfully loaded from temp/Create Account Hash.json
2025-01-26 21:59:42.028 | INFO     | __main__:<module>:3 - b'3981e30acae9fd80ff03e6b1f0338631e32f43c8ca9c0a612fdfd4c458a9b8d9'


2 - Creates a project account into the blockchain;

In [4]:
project_private_key = IrohaCrypto.private_key()
project_public_key = IrohaCrypto.derive_public_key(project_private_key).decode("utf-8")

project_id = generate_project_id()

logger.info(f"Creating project with name: {project_id}")

# project_account = ProjectAccount(
#     project_id=f"{project_id}@{DOMAIN}",
#     public_key=project_public_key
# )

logger.info(f"Creating project with name: {project_id}")

# - Create account
create_account_contract_address = integration_helpers.get_engine_receipts_address(create_account_contract_hash)
create_project_account(create_account_contract_address, project_id, DOMAIN, project_public_key)


2025-01-26 21:59:42.049 | INFO     | __main__:<module>:6 - Creating project with name: 17487
2025-01-26 21:59:42.052 | INFO     | __main__:<module>:13 - Creating project with name: 17487
2025-01-26 21:59:42.053 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-01-26 21:59:42.070 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'3981e30acae9fd80ff03e6b1f0338631e32f43c8ca9c0a612fdfd4c458a9b8d9'
2025-01-26 21:59:42.072 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: FF762AC4E3D64AC77F560CC0DB6D0623F32E8F70
2025-01-26 21:59:42.074 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-01-26 21:59:42.076 | DEBUG    | integration_helpers:tracer:35 - 	Entering "create_project_account"
2025-01-26 21:59:42,114 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-26 21:59:42,117 - INFO - ('ENOUGH_SIGNATURES_COLLECTED',

b'79e0ea2af741d316745af47151e2b34deefce9adb878e2d434d5868e6595a345'

3 - Queries the blockchain to confirm the succesful creation of the account;

In [5]:
#Query - GetAccountDetail

# - Get account info
project_account_hash = get_account(create_account_contract_address, project_id, DOMAIN)
integration_helpers.get_engine_receipts_result(project_account_hash)


2025-01-26 21:59:42.558 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-01-26 21:59:42,561 - WARNING - create_account_contract_address: FF762AC4E3D64AC77F560CC0DB6D0623F32E8F70
2025-01-26 21:59:42,579 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-26 21:59:45,547 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-26 21:59:45,556 - INFO - ('COMMITTED', 5, 0)
2025-01-26 21:59:45,558 - WARNING - Account Data: {'address': 'FF762AC4E3D64AC77F560CC0DB6D0623F32E8F70', 'hex_hash': 'c093bc64a9a4aa8d1b3b9ec97ea3bd6150a0f6b49d43fd4f3e5991c57568498d'}
2025-01-26 21:59:45.559 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-01-26 21:59:45,563 - INFO - Appended data under '17487@test' in logs/account_data.json
2025-01-26 21:59:45.565 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-01-26 21:59:45.567 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_account"
2025-01-26 21:59:45.568 | DEBU

'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00J{"account_id":"17487@test","domain_id":"test","quorum":1,"json_data":"{}"}\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

In [6]:
# Loading the variable from JSON
account_detail_contract_hash = load_variable("Account Detail Contract Hash")
logger.info(account_detail_contract_hash)

2025-01-26 21:59:45,605 - INFO - Variable 'Account Detail Contract Hash' successfully loaded from temp/Account Detail Contract Hash.json
2025-01-26 21:59:45.610 | INFO     | __main__:<module>:3 - b'64b442bbecf02230ecf460f75eadcbbcce3334b831185b81d99e2f3f159a962d'


5 - Sets details for the project account:

In [7]:
# Generate synthetic project metadata JSON (assumed to be done already)
project_metadata_json = generate_synthetic_entry_ld()
account_metadata_cid = upload_json_to_ipfs(project_metadata_json)  # Assuming this is done before calling the function

# Call the function to update or append a new entry to the JSON-LD file
updated_entry_count = update_or_append_project_metadata(
    project_id, 
    project_metadata_json, 
    account_metadata_cid
)


address = integration_helpers.get_engine_receipts_address(account_detail_contract_hash)

# Setting account detail using the generated project metadata CID
hash = set_account_detail(
    address, 
    f"{project_id}@{DOMAIN}",  # Project ID as account
    "account_metadata_cid",    # The key we're setting
    account_metadata_cid       # The value (CID from IPFS)
)

2025-01-26 21:59:45.655 | INFO     | dump_to_json:update_or_append_project_metadata:216 - Updated existing project entry for project ID: 17487@test
2025-01-26 21:59:45.660 | INFO     | dump_to_json:update_or_append_project_metadata:235 - Current total entries in 'datasets/projects.json': 20
2025-01-26 21:59:45.663 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-01-26 21:59:45.682 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'64b442bbecf02230ecf460f75eadcbbcce3334b831185b81d99e2f3f159a962d'
2025-01-26 21:59:45.684 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: ECAB4A3243301E6F3C6EABAC6D6DE1CE7E703118
2025-01-26 21:59:45.685 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-01-26 21:59:45.691 | DEBUG    | integration_helpers:tracer:35 - 	Entering "set_account_detail"


2025-01-26 21:59:45,707 - INFO - None
2025-01-26 21:59:45,712 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-26 21:59:48,580 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-26 21:59:48,587 - INFO - ('COMMITTED', 5, 0)
2025-01-26 21:59:48.589 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_result"
2025-01-26 21:59:48.608 | INFO     | integration_helpers:get_engine_receipts_result:146 - Result data: 000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000001
2025-01-26 21:59:48.609 | DEBUG    | integration_helpers:get_engine_receipts_result:147 - Result decoded string:                                                                                                
2025-01-26 21:59:48.611 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_result"
2025-01-26 21:59:48.612 | DEBUG    | integrat

6 - Queries the blockchain to confirm the succesful details setting

In [8]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, project_id, DOMAIN)
details = integration_helpers.get_engine_receipts_result(get_account_hash)

# Process the raw data
account_metadata_cid = process_raw_data(details) #TO DO - From now

2025-01-26 21:59:48.632 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-01-26 21:59:48,635 - WARNING - create_account_contract_address: FF762AC4E3D64AC77F560CC0DB6D0623F32E8F70
2025-01-26 21:59:48,651 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-26 21:59:48,654 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-26 21:59:51,618 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-26 21:59:51,629 - INFO - ('COMMITTED', 5, 0)
2025-01-26 21:59:51,633 - WARNING - Account Data: {'address': 'FF762AC4E3D64AC77F560CC0DB6D0623F32E8F70', 'hex_hash': '62520fa862719cb57f0b464aeefdc4631785c3bf9bd9593ef421f90c09be33fb'}
2025-01-26 21:59:51.634 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-01-26 21:59:51,640 - INFO - Appended data under '17487@test' in logs/account_data.json
2025-01-26 21:59:51.641 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-01-26 21:59:51.642 | DEBUG    | integration_he

7 - Retrieves Project metadata (JSON-LD formatted) from IPFS

In [9]:
project_metadata = download_json_from_ipfs(account_metadata_cid)
logger.info(f"User Metadata: {project_metadata}")

2025-01-26 21:59:51.698 | INFO     | __main__:<module>:2 - User Metadata: {'@context': {'dc': 'http://purl.org/dc/terms/', 'schema': 'http://schema.org/'}, '@type': 'schema:ResearchProject', 'dc:abstract': 'This study aims to investigate the effects of renewable materials on soil contamination and propose strategies for improvement.', 'schema:endDate': '2028-11-29', 'schema:funding': {'@type': 'schema:Organization', 'schema:name': 'Department of Energy'}, 'schema:keywords': ['renewable materials', 'soil contamination', 'supply chain resilience'], 'schema:location': {'@type': 'schema:Place', 'schema:name': 'Bangkok, Thailand'}, 'schema:name': 'Exploring the Effects of renewable materials on soil contamination', 'schema:startDate': '2018-11-11'}
